# <center>L2 Computational Physics</center>
---

## Week 5: Monte Carlo Methods

In this notebook, you will simulate a system with of three nuclei $A$, $B$ and $C$ where $A$ decays into $B$ and $B$ decays into $C$. If exposed to a neutron flux nucleus $C$ can be activated into a nucleus $A$. 

In [1]:
import numpy
from matplotlib import pyplot as plt
import random

Implement a function that tells whether a transition has occured, based on the transition probability and a random number. Use the random number `r` from `random.random()` and use the procedure described in the notes so that the checks can work in a reproducible way.

In [2]:
def has_transitioned(prob):
    r = random.random()
    if r >= prob:
        a = False
    else:
        a = True
    return a



In [3]:
# this test is worth 1 mark
random.seed(9867)
assert [ has_transitioned(0.5) for i in range(10)] == [False, False, True, False, False, False, False, True, False, True]

We are going to keep track of the state of the atoms using a either `'A'`, `'B'` or `'C'` for each nucleus. For example 
```python
state = ['A', 'A', 'B', 'A', 'C']
```

We will capture the possible physical reaction in a list of allowed transitions such as

`('A', 'B', 0.25 )`

where the first element is the initial state, the second element is the final state of the transition and its probability in the time interval considered is the third argument.

Define a function that takes as arguments the current state and a list of such transition rules and implements the transition (or not) and returns the new state

In [4]:
def evolveOne(currentState, rules):
   
    final = currentState
    for i in range (0, len(rules)):
       
        if currentState == ((rules[i])[0]):
            
            if has_transitioned(rules[i][2]) == False:
                final = rules[i][0]
                
            else:
                final = rules[i][1]
            
                
              
        
    return(final)

In [5]:
# these tests are worth 1 mark
alwaysDecayRules = [
    ('A', 'B', 1.0),
    ('B', 'C', 1.0)
]
assert evolveOne('A', alwaysDecayRules) == 'B'
assert evolveOne('B', alwaysDecayRules) == 'C'

In [6]:
# these tests are worth 2 mark
random.seed(112211)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'B'

assert evolveOne('B', testRules) == 'B'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'

# with no rules there should be no change
assert evolveOne('C', testRules) == 'C'


Now implement a function that takes a list of states and transition them according to the rules passed as argument. This function should return a new vector of states, it should not modify the state passed as an argument!

In [7]:
def evolveMany(states, rules):
    newState = [None] * len(states)
    for i in range(0, len(states)):
        newState[i]= evolveOne(states[i], rules)
    
    
    
    return newState


In [8]:
# these tests are worth 1 mark
random.seed(112287)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
initialTestArray = ['A','B','C']*5
evolvedTest = evolveMany(initialTestArray, testRules)
targetArray = ['B', 'C', 'C', 'A', 'C', 'C', 'A', 'B', 'C', 'A', 'C', 'C', 'B', 'C', 'C'] 
assert evolvedTest == targetArray
# checks the initial array is left unchanged
assert initialTestArray == ['A','B','C']*5

Define a function that evolves a system that starts with initial amounts `NA`, `NB` and `NC` of $A$, $B$ and $C$ nuclei and evolved it in `n_timestep` from time $t=0$ to $t=t_{max}$. The function should return three arrays, one for each atom type, of the number of nuclei of that type at each time step. Each array should contain `n_timestep+1` elements including the initial amount. 
 

In [9]:
def evolve_system(NA, NB, NC, rules, n_step):
    state = (['A'] * NA)+(['B'] * NB)+(['C'] * NC)

   # A_count = numpy.empty(n_step + 1, dtype=int)
   # B_count = numpy.empty(n_step + 1, dtype=int)
   # C_count = numpy.empty(n_step + 1, dtype=int)
    A_count = numpy.zeros(n_step+1)
    B_count = numpy.zeros(n_step+1) 
    C_count = numpy.zeros(n_step+1)
    A_count[0] = NA
    B_count[0] = NB
    C_count[0] = NC

    for i in range(1, n_step+1):
        state = evolveMany(state, rules)
        
        CA = state.count('A')
        CB = state.count('B')
        CC = state.count('C')
        A_count[i] = CA
        B_count[i] = CB
        C_count[i] = CC


    return A_count, B_count, C_count

In [10]:
# these tests are worth 2 marks
rules = [
    ('A', 'B', 0.0033),
    ('B', 'C', 0.0075),
    ('C', 'A', 0.009)
    
]

r1, r2, r3 = evolve_system(0,0,250, rules, 17)
assert len(r1) == 18
assert len(r2) == 18
assert len(r3) == 18

In [11]:
# these tests are worth 2 marks 
testrules = [
    ('A', 'B', 0.086),
    ('B', 'C', 0.075),
    ('C', 'A', 0.19)
    
]

random.seed(9485)
r1, r2, r3 = evolve_system(200,200,200, testrules, 20)
assert (r1 == [200, 213, 233, 250, 258, 251, 266, 263, 259, 260, 265, 259, 256,
        255, 258, 256, 259, 253, 249, 247, 253]).all()
assert (r2 == [200, 198, 201, 206, 205, 214, 214, 212, 216, 221, 225, 234, 236,
        238, 234, 235, 231, 245, 253, 256, 252]).all()
assert (r3 == [200, 189, 166, 144, 137, 135, 120, 125, 125, 119, 110, 107, 108,
        107, 108, 109, 110, 102,  98,  97,  95]).all()

## Plotting tasks

Create a plot with the number of $A$, $B$ and $C$ nuclei, starting with 250 $C$ nuclei and evolving the system for 100 hours using 200 steps and with neutron flux on. Evolve the system for another 100 hours (using 200 steps) without neutron flux (i.e. no transitions from $C$ to $A$).  

The half life of the $A$ atoms is 10.1 hours, the half life of $B$ nuclei is 15.7 hours and we can caracterise the rate of activation of $C$ into $A$ when the neutron flux is on with and effective half-life of 3.2 hours.

The plot should have the appropriate labels and legend. [8 marks]    




Text(0.5,1,'Populations of Nuclei over Time, and Disabling of Neutron Flux at t = 100')

In [12]:
nsteps = 200
t_total = 100
t_half_A = 10.1
t_half_B = 15.7
t_half_C = 3.2

probA = 0.5 / t_half_A
probB = 0.5 / t_half_B
probC = 0.5 / t_half_C

rulesflux = [('A', 'B', probA),('B', 'C', probB),('C', 'A', probC)]
rulesnoflux = [('A', 'B', probA),('B', 'C', probB),('C', 'A', 0)]

f1, f2, f3 = evolve_system(0, 0, 250, rulesflux, 200)
x1 = numpy.linspace(0,200,401)
#plt.plot(x1,f1, label = 'A (Neutron flux on)', color='blue')
#plt.plot(x1,f2, label = 'B (Neutron flux on)', color='red')
#plt.plot(x1,f3, label = 'C (Neutron flux on)', color='black')

i1 = int(f1[-1])
i2 = int(f2[-1])
i3 = int(f3[-1])

n1, n2, n3 = evolve_system(i1, i2, i3, rulesnoflux, 200)

combineda = numpy.empty(401)
combineda[:201] = f1
combineda[200:] = n1

combinedb = numpy.empty(401)
combinedb[:201] = f2
combinedb[200:] = n2

combinedc = numpy.empty(401)
combinedc[:201] = f3
combinedc[200:] = n3

plt.plot(x1,combineda, label = 'A', color='teal')
plt.plot(x1,combinedb, label = 'B', color='orange')
plt.plot(x1,combinedc, label = 'C', color='brown')

#x2 = numpy.linspace(101,201,201)
#plt.plot(x2,n1, label = 'A (Neutron flux off)', color='teal')
#plt.plot(x2,n2, label = 'B (Neutron flux off)', color='orange')
#plt.plot(x2,n3, label = 'C (Neutron flux off)', color='brown')

plt.legend()
plt.xlabel("time (hours)")
plt.ylabel("number of nuclei")
plt.title('Populations of Nuclei over Time, and Disabling of Neutron Flux at t = 100')   

Run the above simulation 20 times with 200 steps and use the results to calculate an average and the uncertainty on the number of $A$ atoms as a function of time. Use and `errorbar` plot for it. You might be interested in the `numpy.average` and `numpy.std` functions. The plot should have axis labels and a title.  [3 marks]


Text(0.5,1,'Populations of A over Time, and Disabling of Neutron Flux at t = 100; 20 simulations')

In [94]:
nsim = 20

nsteps = 200
t_total = 100
t_half_A = 10.1
t_half_B = 15.7
t_half_C = 3.2

probA = 0.5 / t_half_A
probB = 0.5 / t_half_B
probC = 0.5 / t_half_C

rulesflux = [('A', 'B', probA),('B', 'C', probB),('C', 'A', probC)]
rulesnoflux = [('A', 'B', probA),('B', 'C', probB),('C', 'A', 0)]

history = numpy.array(numpy.zeros(nsim), object)

for i in range(0, nsim):
    f1, f2, f3 = evolve_system(0, 0, 250, rulesflux, 200)
    i1 = int(f1[-1])
    i2 = int(f2[-1])
    i3 = int(f3[-1])

    n1, n2, n3 = evolve_system(i1, i2, i3, rulesnoflux, 200)
    combineda = numpy.empty(401)
    combineda[:201] = f1
    combineda[200:] = n1
    history[i] = combineda


acollection = numpy.array(numpy.zeros(20), object)
apoints = numpy.array(numpy.zeros(401), object)
average = numpy.array(numpy.zeros(401), object)
error = numpy.array(numpy.zeros(401), object)
for a in range(0, 401):
        for i in range(0, 20):
            acollection[i] = history[i][a] 
       
        apoints[a] = acollection 
        
        average[a] = numpy.average(apoints[a])
        error[a] = numpy.std(apoints[a])
x1 = numpy.linspace(0,200,401)       
plt.errorbar(x1, average, xerr =  0 , yerr = error , fmt = 'o')  

plt.xlabel("time (hours)")
plt.ylabel("number of nuclei A")
plt.title('Populations of A over Time, and Disabling of Neutron Flux at t = 100; 20 simulations')  
    
